In [41]:
import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
import shutil

In [42]:
carla_root = "/Volumes/ssd_imran/images_out"

In [43]:
dest_path = '/Volumes/ssd_imran/images_without_weather_info'

images = glob.glob(
    os.path.join(
        carla_root,
        '*/*/*.png'
    )
)

for png in tqdm(images):
    out_path = png.replace(carla_root, dest_path).replace(os.path.basename(png), '')
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    shutil.copy(
        png,
        os.path.join(
            out_path,
            "_".join(os.path.basename(png).split('_')[:-1]) + ".png"
        )
    )

100%|██████████| 178124/178124 [09:25<00:00, 314.74it/s]


In [54]:
out_root_path = "/Volumes/ssd_imran/carla_dataset"

for town in os.listdir(dest_path)[0:1]:
    rgb_img = glob.glob(
        os.path.join(
            dest_path,
            town,
            'rgb/*.png'
        )
    )
    for i, img in enumerate(tqdm(rgb_img)):
        try:
            if i < len(rgb_img)/2:
                for sub_dir in os.listdir(os.path.join(dest_path, town)):
                    if sub_dir.startswith('.') or sub_dir == 'opt_flow':
                        continue
                    out_path_t = os.path.join(out_root_path, 'training/temp_all')
                    out_path = img.replace(dest_path, out_path_t).replace(os.path.basename(img), '').replace(f'{town}/', '').replace('rgb', sub_dir)
                    if not os.path.exists(out_path):
                        os.makedirs(out_path)
                    shutil.copy(
                        img.replace('rgb', sub_dir),
                        os.path.join(
                            out_path,
                            f"{town}_" + "_".join(os.path.basename(img).split('_')[:1]) + ".png"
                        )
                    )
            else:
                for sub_dir in os.listdir(os.path.join(dest_path, town)):
                    if sub_dir.startswith('.') or sub_dir == 'opt_flow':
                        continue
                    out_path_t = os.path.join(out_root_path, 'testing')
                    out_path = img.replace(dest_path, out_path_t).replace(os.path.basename(img), '').replace(f'{town}/', '').replace('rgb', sub_dir)
                    if not os.path.exists(out_path):
                        os.makedirs(out_path)
                    shutil.copy(
                        img.replace('rgb', sub_dir),
                        os.path.join(
                            out_path,
                            f"{town}_" + "_".join(os.path.basename(img).split('_')[:1]) + ".png"
                        )
                    )
        except Exception as e:
            del_pth = os.path.join(
                img.replace(dest_path, os.path.join(out_root_path, 'training/temp_all')).replace(os.path.basename(img), '').replace(f'{town}/', ''),
                f"{town}_" + "_".join(os.path.basename(img).split('_')[:1]) + ".png"
            )
            if os.path.exists(del_pth):
                os.remove(
                    del_pth
                )
            # print(e)


100%|██████████| 3642/3642 [01:04<00:00, 56.23it/s]


In [37]:
video_count = 0
weather_to_vid_frame_map = {}


for town in os.listdir(carla_root):
    current_weather = ''
    previous_weather = ''
    rgb_img = glob.glob(
        os.path.join(
            carla_root,
            town,
            'rgb/*.png'
        )
    )
    rgb_img.sort()
    for i, img in enumerate(tqdm(rgb_img)):
        find_in = 0

        current_weather = os.path.basename(img).replace('.png', '').split('_')[-1]
        new_vid = False
        if current_weather != previous_weather:
            previous_weather = current_weather
            video_count += 1
            new_vid = True
        if i == 0 or new_vid:
            # print(i)
            key = str(video_count).zfill(3)
            if key not in weather_to_vid_frame_map.keys():
                weather_to_vid_frame_map[key] = []
            weather_to_vid_frame_map[key].append(img)
        else:
            img_num_i = int(os.path.basename(img).replace('.png', '').split('_')[0])
            img_num_i_m_1 = int(os.path.basename(rgb_img[i-1]).replace('.png', '').split('_')[0])
            # print(i, img_num_i, img_num_i_m_1, img_num_i-img_num_i_m_1)
            if img_num_i - img_num_i_m_1 <= 6:
                key = str(video_count).zfill(3)
                if key not in weather_to_vid_frame_map.keys():
                    weather_to_vid_frame_map[key] = []
                weather_to_vid_frame_map[key].append(img)
            else:
                video_count += 1
                key = str(video_count).zfill(3)
                if key not in weather_to_vid_frame_map.keys():
                    weather_to_vid_frame_map[key] = []
                weather_to_vid_frame_map[key].append(img)

        previous_weather = current_weather

100%|██████████| 3159/3159 [00:00<00:00, 366989.98it/s]


In [40]:
for key in weather_to_vid_frame_map.keys():
    print(key, len(weather_to_vid_frame_map[key]), list(weather_to_vid_frame_map[key])[0], list(weather_to_vid_frame_map[key])[-1])


001 167 /Volumes/ssd_imran/images_out/town_02/rgb/00001692_fl_1000_0_Clear Noon.png /Volumes/ssd_imran/images_out/town_02/rgb/00002688_fl_1000_0_Clear Noon.png
002 265 /Volumes/ssd_imran/images_out/town_02/rgb/00002694_fl_1000_0_Clear Sunset.png /Volumes/ssd_imran/images_out/town_02/rgb/00004278_fl_1000_0_Clear Sunset.png
003 196 /Volumes/ssd_imran/images_out/town_02/rgb/00004284_fl_1000_0_Cloudy Night.png /Volumes/ssd_imran/images_out/town_02/rgb/00005454_fl_1000_0_Cloudy Night.png
004 171 /Volumes/ssd_imran/images_out/town_02/rgb/00005460_fl_1000_0_Cloudy Noon.png /Volumes/ssd_imran/images_out/town_02/rgb/00006480_fl_1000_0_Cloudy Noon.png
005 81 /Volumes/ssd_imran/images_out/town_02/rgb/00006486_fl_1000_0_Cloudy Sunset.png /Volumes/ssd_imran/images_out/town_02/rgb/00006966_fl_1000_0_Cloudy Sunset.png
006 134 /Volumes/ssd_imran/images_out/town_02/rgb/00006972_fl_1000_0_Default.png /Volumes/ssd_imran/images_out/town_02/rgb/00007770_fl_1000_0_Default.png
007 388 /Volumes/ssd_imran/imag

In [55]:
import glob
import random
import os

root = '/Volumes/ssd_imran/carla_dataset/training/temp_all'
out_p = "/Volumes/ssd_imran/carla_dataset/training"

images = glob.glob(os.path.join(root, 'rgb/*.png'))

train_indices = random.sample(range(0, len(images)-1), int(len(images)*0.8))

for i, rgb in enumerate(tqdm(images)):
    if i in train_indices:
        shutil.copy(rgb, os.path.join(out_p, 'train', 'images'))
        shutil.copy(rgb.replace('rgb', 'sem_seg_raw'), os.path.join(out_p, 'train', 'labels'))
        shutil.copy(rgb.replace('rgb', 'depth'), os.path.join(out_p, 'train', 'depth'))
    else:
        shutil.copy(rgb, os.path.join(out_p, 'validation', 'images'))
        shutil.copy(rgb.replace('rgb', 'sem_seg_raw'), os.path.join(out_p, 'validation', 'labels'))
        shutil.copy(rgb.replace('rgb', 'depth'), os.path.join(out_p, 'validation', 'depth'))

100%|██████████| 12133/12133 [02:45<00:00, 73.49it/s]
